# Unique Plate Stability - 01 Machine Learning - Data Preparation

## Setup

### Confidentiality

**All information in this document is strictly confidiental**
**Copyright (C) 2019 HES-SO Valais-Wallis - All Rights Reserved**

### Import sub-modules

In [398]:
# Import required sub-modules

# python
import sys
import os
import enum
import datetime

# iPython
import IPython
from IPython.display import display
from IPython.display import Image

# pandas
import pandas as pd

# numpy
import numpy as np

# plotly
import plotly as ply
import plotly.figure_factory as ff
ply.offline.init_notebook_mode(connected=True)
import plotly.io as pio

# scikit
import sklearn as sklearn
#from sklearn import preprocessing
#from sklearn.tree import DecisionTreeRegressor

# watermark
import watermark
%load_ext watermark
%watermark -a "Silvan Zahno" -d -v -iv -m -h

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
numpy     1.16.2
plotly    3.8.1
sklearn   0.20.3
IPython   7.4.0
watermark 1.8.1
pandas    0.24.2
Silvan Zahno 2019-07-02 

CPython 3.7.3
IPython 7.4.0

compiler   : MSC v.1915 64 bit (AMD64)
system     : Windows
release    : 10
machine    : AMD64
processor  : Intel64 Family 6 Model 142 Stepping 10, GenuineIntel
CPU cores  : 8
interpreter: 64bit
host name  : WE6996


### Configurations

In [399]:
# Setup local input directories
inputDir = "input/"

if (os.path.exists(inputDir)) is False:
    os.makedirs(inputDir)
if (os.path.isdir(inputDir)) is False:
    raise NotADirectoryError("{} is not a directory".format(inputDir))

In [400]:
# Setup local input directories
outputDir = "output/"

if (os.path.exists(outputDir)) is False:
    os.makedirs(outputDir)
if (os.path.isdir(outputDir)) is False:
    raise NotADirectoryError("{} is not a directory".format(outputDir))

In [401]:
# Graph output Options
class GraphOutputOption(enum.Enum):
    none = 'none'                     # Do not generate any plots
    inline = 'inline'                 # Generate inline plots only
    htmlFile = 'htmlFile'             # Generate plots in external HTML files
    both = 'both'                     # Generate plots both inline and in external html files
    
class GraphInteractionOption(enum.Enum):
    static = 'static'                 # Generate static inline plots (as images)
    interactive = 'interactive'       # Generate interactive inline plots

notebookGraphingInteraction = GraphInteractionOption('static')
notebookGraphingOutputs = GraphOutputOption('both')

staticImageSize = {'width':1000, 'height':500, 'scale':1}

GraphAutoOpenHTML = False              # Auto open external HTML files [True/False]

In [402]:
# Pandas output options
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 70)
pd.set_option('display.width', 200)

### Common functions

## Data Import

In [403]:
dirlist = os.listdir(inputDir)
print("Files from directory ", inputDir)
for file in dirlist:
  print("  * ",file)


Files from directory  input/
  *  .ipynb_checkpoints
  *  Data process USP v1_1.xlsx
  *  Data process USP v1_2.xlsx
  *  ExportArseneFiltreFinal20190301 v1_2.csv
  *  ExportArseneFiltreFinal20190301 v1_2_1.csv
  *  ExportArseneFiltreFinal20190301.xlsx


### CSV to pandas DF

In [404]:
# Import CSV files into a pandas dataframe
uspExcelExportFileName = 'ExportArseneFiltreFinal20190301 v1_2_1.csv'
uspExcelExportFilePath = inputDir + uspExcelExportFileName
rawUspDf = pd.read_csv(uspExcelExportFilePath, sep=';')

rawUspDf.head()

,SK_LOT,SK_TRACE,OF,BARRE,PLATEAU,LOT,ALLIAGE_INTERNE,ETAT,EPAISSEUR_FINALE,SOUSFAMILLE,FOUR_COULEE,COMPO_SI,COMPO_FE,COMPO_CU,COMPO_MN,COMPO_MG,COMPO_CR,COMPO_NI,COMPO_ZN,COMPO_TI,COMPO_ZR,PRECHAUFFAGE_NUMERO_FOUR,PRECHAUFFAGE_POSITION_FOUR,PRECHAUFFAGE_T_C_AIR_MAXI,TR_DATE_POTENCE,TR_T_C_POTENCE,TR_EPAISSEUR_POTENCE,TR_NOMBRE_EBAUCHE,TR_NOMBRE_PASSE,TR_EPAISSEUR_DERNIERE_PASSE,TR_VITESSE_DERNIERE_PASSE,TR_T_C_DERNIERE_PASSE,TR_VITESSE_ENGAGEMENT_DERNIERE_PASSE,TR_TPS_POTENCE_PREMIERE_PASSE,TR_REDUCTION_EPAISSEUR_DERNIERE_PASSE,...,BARRE_LONGUEUR,BARRE_LARGEUR,BARRE_EPAISSEUR,TREMPE_MACHINE,TREMPE_SK_TEMPS,TREMPE_DATE_DEBUT,TREMPE_DATE_FIN,TREMPE_RECETTE,TRAC_MACHINE,TRAC_SK_TEMPS,TRAC_SENS_TRACTION,TRAC_TXPREML,TRAC_TXPREMT,TRAC_TXDERNL,TRAC_TXDERNT,TRAC_PLNTRANS,TRAC_PLNLONG,TRAC_PLNTRANSTETE,TRAC_PLNTRANSMILIEU,TRAC_PLNTRANSPIED,TRAC_EPMINL,TRAC_EPMINT,TRAC_EPMAXL,TRAC_EPMAXT,REVENU_MACHINE,REVENU_DATE_DEBUT,REVENU_NOCHARGE,LABO_RM,LABO_RP02,LABO_ALGMT,RASPTAB_SK_TEMPS,RASPTAB_ECHANTILLON,RASPTAB_TIL_VAL,RASPTAB_TIT_VAL,RASPTAB_VAL_MAX_TIL_TIT
0,1208407,141800,6278466,5313129152,2494272415,U4272401,7213,T651,20.50,31,HG_TP09,0.09,0.17,1.54,0.048,2.29,0.18,0.005,5.9,0.026,0.023,F13,5,296.3,17.02.2016 04:03,395,355.0,1,20,21.42,-1136.7,422.58,-503.4,10.66,12.26,...,2500,1650,380,FTRE07,20160223,20160223010719,20160223013704,R7511,TRAC06,20160223,L,0.0,0,0.0,0,0.0,0.0,0.7,0.7,1.9,20.52,0,20.63,0,FAIR25,20160226020754,20165102.0,576.2,514.3,13.4,20160308,U4272401-J,508,78,1100
1,1208408,141800,6278466,5313129152,2494272415,U4272402,7213,T651,20.50,31,HG_TP09,0.09,0.17,1.54,0.048,2.29,0.18,0.005,5.9,0.026,0.023,F13,5,296.3,17.02.2016 04:03,395,355.0,1,20,21.42,-1136.7,422.58,-503.4,10.66,12.26,...,2500,1650,380,FTRE07,20160223,20160223010719,20160223013704,R7511,TRAC06,20160223,L,0.0,0,0.0,0,0.0,0.0,0.7,0.7,0.7,20.53,0,20.60,0,FAIR25,20160226020754,20165102.0,577.6,515.7,12.9,20160308,U4272402-J,589,159,1100
2,1208409,141800,6278466,5313129152,2494272415,U4272403,7213,T651,20.50,31,HG_TP09,0.09,0.17,1.54,0.048,2.29,0.18,0.005,5.9,0.026,0.023,F13,5,296.3,17.02.2016 04:03,395,355.0,1,20,21.42,-1136.7,422.58,-503.4,10.66,12.26,...,2500,1650,380,FTRE07,20160223,20160223013536,20160223020456,R7511,TRAC06,20160223,L,0.0,0,0.0,0,0.0,0.0,0.7,0.7,0.7,20.51,0,20.59,0,FAIR25,20160226020754,20165102.0,576.0,513.7,12.9,20160308,U4272403-J,597,104,1100
3,1208410,141800,6278466,5313129152,2494272415,U4272404,7213,T651,20.50,31,HG_TP09,0.09,0.17,1.54,0.048,2.29,0.18,0.005,5.9,0.026,0.023,F13,5,296.3,17.02.2016 04:03,395,355.0,1,20,21.42,-1136.7,422.58,-503.4,10.66,12.26,...,2500,1650,380,FTRE07,20160223,20160223013536,20160223020456,R7511,TRAC06,20160223,L,0.0,0,0.0,0,0.0,0.0,0.7,0.7,0.7,20.50,0,20.61,0,FAIR25,20160226020754,20165102.0,579.1,516.6,13.9,20160308,U4272404-J,630,114,1100
4,1206475,141800,6278465,5313129122,2494272715,U4272701,7213,T651,22.23,31,HG_TP09,0.09,0.17,1.54,0.048,2.29,0.18,0.005,5.9,0.026,0.023,F47,6,307.1,14.02.2016 14:32,387,356.0,1,20,23.21,-825.2,397.75,-507.4,93.57,10.69,...,2500,1650,380,FTRE07,20160222,20160222143423,20160222150536,R7511,TRAC06,20160222,L,0.0,0,0.0,0,0.0,0.0,0.7,0.6,0.5,22.20,0,22.39,0,FAIR25,20160226020754,20165102.0,573.8,512.1,12.8,20160309,U4272701-J,954,79,850


### Count the null values in the Data Table

In [405]:
verboseCell = 1
# Check if we have any empty field in the data
if len(np.where(pd.isnull(rawUspDf))[0]) != 0:
    if verboseCell >= 0:
      print('WARNING!!: Dataframe has null (NaN or empty) cells')
    for column in rawUspDf:
        listofNullCells = np.where(pd.isnull(rawUspDf[column]))
        lenNullCells = len(listofNullCells[0])
        if lenNullCells != 0:
          if verboseCell >= 1:
            print("  * Column '{}' has {} null cells".format(column, lenNullCells))
          if verboseCell >= 2:  
            print("      {}".format(listofNullCells))
else:
    print("All good: no empty cells in Dataframe")

WARNING!!: Dataframe has null (NaN or empty) cells
  * Column 'TR_TPS_POTENCE_PREMIERE_PASSE' has 368 null cells
  * Column 'TREMPE_RECETTE' has 37 null cells
  * Column 'REVENU_NOCHARGE' has 1918 null cells


In [406]:
# dropna drops missing values
if len(np.where(pd.isnull(rawUspDf))[0]) != 0:
  len_with_na = len(rawUspDf)
  cleanedUspDf = rawUspDf.dropna(axis=0)
  len_without_na = len(cleanedUspDf)
  print("Remove NA-Values from Table")
  print("  * {} Rows found".format(len_with_na))
  print("  * {} Rows with NA-Values".format(len_with_na-len_without_na))
  print("  * {} Rows without NA-Values available".format(len_without_na))
else:
  print("No duplicates found")
  
  

Remove NA-Values from Table
  * 7243 Rows found
  * 2237 Rows with NA-Values
  * 5006 Rows without NA-Values available


### Clean Duplicates

In [407]:
if (len(cleanedUspDf[cleanedUspDf.duplicated()]) > 0):
  print("Duplicates found within the Table")
  len_with_duplicates = len(cleanedUspDf)
  cleanedUspDf = cleanedUspDf.drop_duplicates()
  len_without_duplicates = len(cleanedUspDf)
  print("Remove Duplicates from Table".format(len_with_duplicates))
  print("  * {} Elements found".format(len_with_duplicates))
  print("  * {} Duplicates".format(len_with_duplicates - len_without_duplicates))
  print("  * {} Unique Elements available ".format(len_without_duplicates))
else:
  print("No Duplicates found")

No Duplicates found


### Drop Unused columns

In [408]:
# Drop all columns with only one distinct value
for col in cleanedUspDf.columns:
    if len(cleanedUspDf[col].unique()) == 1:
        print("Drop {} only value \"{}\" present".format(col, cleanedUspDf[col][0]))
        cleanedUspDf.drop(col,inplace=True,axis=1)

Drop ETAT only value "T651" present
Drop SOUSFAMILLE only value "31" present
Drop BARRE_EPAISSEUR only value "380" present
Drop TRAC_SENS_TRACTION only value "L" present
Drop TRAC_TXPREMT only value "0" present
Drop TRAC_TXDERNT only value "0" present
Drop TRAC_EPMINT only value "0" present
Drop TRAC_EPMAXT only value "0" present


In [409]:
# Drop specific columns which are not necessary
cleanedUspDf.drop("SK_LOT",inplace=True,axis=1)              # Not needed only traceability
cleanedUspDf.drop("SK_TRACE",inplace=True,axis=1)            # Not needed only traceability
cleanedUspDf.drop("PLATEAU",inplace=True,axis=1)             # Same as BARRE
cleanedUspDf.drop("LOT",inplace=True,axis=1)                 # Not needed only traceability
cleanedUspDf.drop("RASPTAB_ECHANTILLON",inplace=True,axis=1) # Not needed only traceability

### Convert datetime

In [410]:
cleanedUspDf['TR_DATE_POTENCE'] =  pd.to_datetime(cleanedUspDf['TR_DATE_POTENCE'], format='%d.%m.%Y %H:%M')
cleanedUspDf['TREMPE_SK_TEMPS'] =  pd.to_datetime(cleanedUspDf['TREMPE_SK_TEMPS'], format='%Y%m%d')
cleanedUspDf['TREMPE_DATE_DEBUT'] =  pd.to_datetime(cleanedUspDf['TREMPE_DATE_DEBUT'], format='%Y%m%d%H%M%S')
cleanedUspDf['TREMPE_DATE_FIN'] =  pd.to_datetime(cleanedUspDf['TREMPE_DATE_FIN'], format='%Y%m%d%H%M%S')
cleanedUspDf['TRAC_SK_TEMPS'] =  pd.to_datetime(cleanedUspDf['TRAC_SK_TEMPS'], format='%Y%m%d')
cleanedUspDf['REVENU_DATE_DEBUT'] =  pd.to_datetime(cleanedUspDf['REVENU_DATE_DEBUT'], format='%Y%m%d%H%M%S')
cleanedUspDf['RASPTAB_SK_TEMPS'] =  pd.to_datetime(cleanedUspDf['RASPTAB_SK_TEMPS'], format='%Y%m%d')

### Calc timestamps

In [411]:
# Perform duration calculation with the given timestamps
# TODO

## Statistical Analysis

In [412]:
desc = cleanedUspDf.describe(percentiles=[0.25, 0.5, 0.75])
desc

,OF,BARRE,ALLIAGE_INTERNE,EPAISSEUR_FINALE,COMPO_SI,COMPO_FE,COMPO_CU,COMPO_MN,COMPO_MG,COMPO_CR,COMPO_NI,COMPO_ZN,COMPO_TI,COMPO_ZR,PRECHAUFFAGE_POSITION_FOUR,PRECHAUFFAGE_T_C_AIR_MAXI,TR_T_C_POTENCE,TR_EPAISSEUR_POTENCE,TR_NOMBRE_EBAUCHE,TR_NOMBRE_PASSE,TR_EPAISSEUR_DERNIERE_PASSE,TR_VITESSE_DERNIERE_PASSE,TR_T_C_DERNIERE_PASSE,TR_VITESSE_ENGAGEMENT_DERNIERE_PASSE,TR_TPS_POTENCE_PREMIERE_PASSE,TR_REDUCTION_EPAISSEUR_DERNIERE_PASSE,TR_REDUCTION_EPAISSEUR_AVANT_DERNIERE_PASSE,TR_REDUCTION_EPAISSEUR_AVANT_AVANT_DERNIERE_PASSE,TR_REDUCTION_EPAISSEUR_MOYENNE,BARRE_LONGUEUR,BARRE_LARGEUR,TRAC_TXPREML,TRAC_TXDERNL,TRAC_PLNTRANS,TRAC_PLNLONG,TRAC_PLNTRANSTETE,TRAC_PLNTRANSMILIEU,TRAC_PLNTRANSPIED,TRAC_EPMINL,TRAC_EPMAXL,REVENU_NOCHARGE,LABO_RM,LABO_RP02,LABO_ALGMT,RASPTAB_TIL_VAL,RASPTAB_TIT_VAL,RASPTAB_VAL_MAX_TIL_TIT
count,5.006000e+03,5.006000e+03,5006.000000,5006.000000,5006.000000,5006.000000,5006.000000,5006.000000,5006.000000,5006.000000,5006.000000,5006.000000,5006.000000,5006.000000,5006.000000,5006.000000,5006.000000,5006.000000,5006.000000,5006.000000,5006.000000,5006.000000,5006.000000,5006.000000,5006.000000,5006.000000,5006.000000,5006.000000,5006.000000,5006.000000,5006.000000,5006.000000,5006.000000,5006.000000,5006.000000,5006.000000,5006.000000,5006.000000,5006.000000,5006.000000,5.006000e+03,5006.000000,5006.000000,5006.000000,5006.000000,5006.000000,5006.000000
mean,6.289439e+06,6.941100e+09,7214.322014,22.224071,0.087461,0.149632,1.473155,0.048992,2.406689,0.186141,0.005211,5.769209,0.026101,0.018094,4.432481,383.171554,390.036756,353.828006,1.494407,23.546544,23.227219,-1009.600499,370.312799,-309.644367,166.583034,7.855491,9.578546,12.509642,14.082305,2483.949261,1617.119457,1.403987,0.784179,0.526988,0.321510,0.483320,0.582541,0.482058,16.567669,16.623767,2.017394e+07,576.934547,511.611025,12.532881,543.722133,251.221334,930.803036
std,7.566079e+03,7.987908e+08,0.901205,7.369149,0.014143,0.017026,0.054569,0.010341,0.125393,0.004226,0.001672,0.172383,0.003305,0.004564,2.463750,81.010954,6.727569,1.761896,0.500019,1.568207,8.117834,264.606216,17.549683,193.279381,93.224256,2.666988,3.379950,3.839478,0.680393,237.070374,80.150785,0.746769,0.895939,0.836391,0.460638,0.464994,0.600974,0.477904,13.794638,13.835471,8.222421e+03,12.869320,14.769666,0.855534,259.291914,194.570382,347.964015
min,6.277498e+06,5.296229e+09,7213.000000,13.000000,0.039000,0.080000,1.289100,0.004000,2.150000,0.180000,0.002200,5.133100,0.014000,0.000600,1.000000,0.000000,369.000000,326.000000,1.000000,18.000000,13.570000,-2836.000000,244.670000,-2750.100000,10.660000,1.800000,1.810000,2.570000,8.970000,1800.000000,1400.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.016006e+07,496.610000,339.050000,7.700000,16.000000,2.000000,400.000000
25%,6.283235e+06,6.185129e+09,7213.000000,15.000000,0.079400,0.140000,1.444100,0.043000,2.270000,0.183000,0.004400,5.740000,0.024000,0.015100,2.000000,314.200000,386.000000,353.000000,1.000000,23.000000,15.690000,-1149.200000,365.630000,-392.900000,85.155000,5.790000,7.030000,9.100000,13.920000,2350.000000,1650.000000,1.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.016544e+07,568.000000,502.592500,12.000000,351.000000,101.000000,650.000000
50%,6.288836e+06,7.070129e+09,7215.000000,20.000000,0.088400,0.150000,1.477000,0.050000,2.470000,0.186400,0.005000,5.800000,0.026500,0.018000,4.000000,444.000000,390.000000,354.000000,1.000000,24.000000,20.880000,-1129.000000,371.230000,-381.850000,172.600000,7.275000,8.270000,13.240000,14.100000,2500.000000,1650.000000,1.700000,0.000000,0.000000,0.000000,0.500000,0.500000,0.500000,20.290000,20.360000,2.017523e+07,577.800000,512.465000,12.500000,510.500000,198.000000,1100.000000
75%,6.294644e+06,7.296229e+09,7215.000000,25.500000,0.095200,0.160000,1.510000,0.055900,2.510000,0.190000,0.005600,5.860000,0.028100,0.021200,6.000000,451.000000,394.000000,355.000000,2.000000,24.000000,26.470000,-902.50000

## Prepare for Machine learning

### Filter by thickness

In [413]:
print(np.sort(cleanedUspDf["EPAISSEUR_FINALE"].unique()))
print("There are {} different thicknesses".format(len(cleanedUspDf["EPAISSEUR_FINALE"].unique())))

[13.    14.    15.    15.875 15.88  16.    17.    18.    18.5   19.05
 20.    20.5   21.    21.5   22.    22.2   22.225 22.23  23.    25.
 25.4   25.5   26.    27.    28.    28.57  28.575 29.    30.    31.
 31.75  32.    35.    35.5   36.    38.    38.1   39.    39.7   40.   ]
There are 40 different thicknesses


In [414]:
filteredDfs = []
for thickness in cleanedUspDf["EPAISSEUR_FINALE"].unique():
    filteredDfs.append(cleanedUspDf[cleanedUspDf["EPAISSEUR_FINALE"] == thickness])

for df in filteredDfs:
    print("Thickness {:6} has {:6} elements".format(df["EPAISSEUR_FINALE"].values[0] ,len(df)))

Thickness   20.5 has     34 elements
Thickness  22.23 has     28 elements
Thickness   20.0 has    712 elements
Thickness   16.0 has    328 elements
Thickness   32.0 has     63 elements
Thickness   40.0 has    232 elements
Thickness   15.0 has   1284 elements
Thickness   25.0 has    386 elements
Thickness   35.0 has    213 elements
Thickness  19.05 has     29 elements
Thickness   30.0 has    433 elements
Thickness   25.5 has    194 elements
Thickness   23.0 has    309 elements
Thickness   22.0 has    105 elements
Thickness   36.0 has     27 elements
Thickness   35.5 has     17 elements
Thickness   18.5 has      8 elements
Thickness   14.0 has     20 elements
Thickness   39.0 has      4 elements
Thickness   31.0 has     34 elements
Thickness  31.75 has     22 elements
Thickness  15.88 has     30 elements
Thickness 22.225 has     25 elements
Thickness   13.0 has    146 elements
Thickness   26.0 has     40 elements
Thickness   21.0 has     75 elements
Thickness   27.0 has     25 elements
T

### Select Prediction Model

In [415]:
filteredDf = filteredDfs[6]

In [416]:
# LABO_RM
# LABO_RP02
# LABO_ALGMT
# RASPTAB_TIL_VAL
# RASPTAB_TIT_VAL
usp_response = ['RASPTAB_TIT_VAL']
y = filteredDf['RASPTAB_TIT_VAL']

### Choosing Features

In [417]:
filteredDf["EPAISSEUR_FINALE"].values[0]

15.0

In [418]:
filteredDf.columns

Index(['OF', 'BARRE', 'ALLIAGE_INTERNE', 'EPAISSEUR_FINALE', 'FOUR_COULEE', 'COMPO_SI', 'COMPO_FE', 'COMPO_CU', 'COMPO_MN', 'COMPO_MG', 'COMPO_CR', 'COMPO_NI', 'COMPO_ZN', 'COMPO_TI', 'COMPO_ZR',
       'PRECHAUFFAGE_NUMERO_FOUR', 'PRECHAUFFAGE_POSITION_FOUR', 'PRECHAUFFAGE_T_C_AIR_MAXI', 'TR_DATE_POTENCE', 'TR_T_C_POTENCE', 'TR_EPAISSEUR_POTENCE', 'TR_NOMBRE_EBAUCHE', 'TR_NOMBRE_PASSE',
       'TR_EPAISSEUR_DERNIERE_PASSE', 'TR_VITESSE_DERNIERE_PASSE', 'TR_T_C_DERNIERE_PASSE', 'TR_VITESSE_ENGAGEMENT_DERNIERE_PASSE', 'TR_TPS_POTENCE_PREMIERE_PASSE',
       'TR_REDUCTION_EPAISSEUR_DERNIERE_PASSE', 'TR_REDUCTION_EPAISSEUR_AVANT_DERNIERE_PASSE', 'TR_REDUCTION_EPAISSEUR_AVANT_AVANT_DERNIERE_PASSE', 'TR_REDUCTION_EPAISSEUR_MOYENNE', 'BARRE_LONGUEUR',
       'BARRE_LARGEUR', 'TREMPE_MACHINE', 'TREMPE_SK_TEMPS', 'TREMPE_DATE_DEBUT', 'TREMPE_DATE_FIN', 'TREMPE_RECETTE', 'TRAC_MACHINE', 'TRAC_SK_TEMPS', 'TRAC_TXPREML', 'TRAC_TXDERNL',
       'TRAC_PLNTRANS', 'TRAC_PLNLONG', 'TRAC_PLNTRANSTETE',

In [419]:
filteredDf.head()

,OF,BARRE,ALLIAGE_INTERNE,EPAISSEUR_FINALE,FOUR_COULEE,COMPO_SI,COMPO_FE,COMPO_CU,COMPO_MN,COMPO_MG,COMPO_CR,COMPO_NI,COMPO_ZN,COMPO_TI,COMPO_ZR,PRECHAUFFAGE_NUMERO_FOUR,PRECHAUFFAGE_POSITION_FOUR,PRECHAUFFAGE_T_C_AIR_MAXI,TR_DATE_POTENCE,TR_T_C_POTENCE,TR_EPAISSEUR_POTENCE,TR_NOMBRE_EBAUCHE,TR_NOMBRE_PASSE,TR_EPAISSEUR_DERNIERE_PASSE,TR_VITESSE_DERNIERE_PASSE,TR_T_C_DERNIERE_PASSE,TR_VITESSE_ENGAGEMENT_DERNIERE_PASSE,TR_TPS_POTENCE_PREMIERE_PASSE,TR_REDUCTION_EPAISSEUR_DERNIERE_PASSE,TR_REDUCTION_EPAISSEUR_AVANT_DERNIERE_PASSE,TR_REDUCTION_EPAISSEUR_AVANT_AVANT_DERNIERE_PASSE,TR_REDUCTION_EPAISSEUR_MOYENNE,BARRE_LONGUEUR,BARRE_LARGEUR,TREMPE_MACHINE,TREMPE_SK_TEMPS,TREMPE_DATE_DEBUT,TREMPE_DATE_FIN,TREMPE_RECETTE,TRAC_MACHINE,TRAC_SK_TEMPS,TRAC_TXPREML,TRAC_TXDERNL,TRAC_PLNTRANS,TRAC_PLNLONG,TRAC_PLNTRANSTETE,TRAC_PLNTRANSMILIEU,TRAC_PLNTRANSPIED,TRAC_EPMINL,TRAC_EPMAXL,REVENU_MACHINE,REVENU_DATE_DEBUT,REVENU_NOCHARGE,LABO_RM,LABO_RP02,LABO_ALGMT,RASPTAB_SK_TEMPS,RASPTAB_TIL_VAL,RASPTAB_TIT_VAL,RASPTAB_VAL_MAX_TIL_TIT
34,6277589,5335229152,7215,15.0,HG_TP09,0.1,0.18,1.49,0.053,2.47,0.19,0.0053,5.74,0.027,0.017,F49,10,322.9,2015-12-12 19:04:00,395,354.0,2,25,15.66,-1143.1,373.71,-530.3,104.12,3.95,4.95,6.08,13.53,2500,1650,FTRE06,2016-01-07,2016-01-07 02:47:02,2016-01-07 03:05:16,R4675,TRAC04,2016-01-07,1.7,1.7,2.7,0.8,0.0,0.0,0.0,0.0,0.0,FAIR09,2016-01-10 11:05:29,20165009.0,584.10,508.13,11.3,2016-01-21,191,259,1300
35,6277589,5335229152,7215,15.0,HG_TP09,0.1,0.18,1.49,0.053,2.47,0.19,0.0053,5.74,0.027,0.017,F49,10,322.9,2015-12-12 19:04:00,395,354.0,2,25,15.66,-1143.1,373.71,-530.3,104.12,3.95,4.95,6.08,13.53,2500,1650,FTRE06,2016-01-07,2016-01-07 03:05:51,2016-01-07 03:24:10,R4675,TRAC04,2016-01-07,1.7,1.7,1.3,0.8,0.0,0.0,0.0,0.0,0.0,FAIR09,2016-01-10 11:05:29,20165009.0,588.71,518.82,12.6,2016-01-21,332,207,1300
36,6277589,5335229152,7215,15.0,HG_TP09,0.1,0.18,1.49,0.053,2.47,0.19,0.0053,5.74,0.027,0.017,F49,10,322.9,2015-12-12 19:04:00,395,354.0,2,25,15.66,-1143.1,373.71,-530.3,104.12,3.95,4.95,6.08,13.53,2500,1650,FTRE06,2016-01-07,2016-01-07 02:51:11,2016-01-07 03:13:18,R4675,TRAC04,2016-01-07,1.7,1.7,1.3,0.8,0.0,0.0,0.0,0.0,0.0,FAIR09,2016-01-10 11:05:29,20165009.0,588.71,518.82,12.6,2016-01-21,332,207,1300
37,6277589,5335229152,7215,15.0,HG_TP09,0.1,0.18,1.49,0.053,2.47,0.19,0.0053,5.74,0.027,0.017,F49,10,322.9,2015-12-12 19:04:00,395,354.0,2,25,15.66,-1143.1,373.71,-530.3,104.12,3.95,4.95,6.08,13.53,2500,1650,FTRE06,2016-01-07,2016-01-07 03:51:26,2016-01-07 04:09:46,R4675,TRAC04,2016-01-07,1.7,1.7,1.2,0.8,0.0,0.0,0.0,0.0,0.0,FAIR09,2016-01-10 11:05:29,20165009.0,588.29,523.52,12.9,2016-01-21,1407,275,1300
38,6277589,5335229152,7215,15.0,HG_TP09,0.1,0.18,1.49,0.053,2.47,0.19,0.0053,5.74,0.027,0.017,F49,10,322.9,2015-12-12 19:04:00,395,354.0,2,25,15.66,-1143.1,373.71,-530.3,104.12,3.95,4.95,6.08,13.53,2500,1650,FTRE06,2016-01-07,2016-01-07 03:24:44,2016-01-07 03:43:03,R4675,TRAC04,2016-01-07,1.7,1.7,1.0,0.8,0.0,0.0,0.0,0.0,0.0,FAIR09,2016-01-10 11:05:29,20165009.0,581.69,509.07,12.3,2016-01-21,1026,39,1300


In [420]:
cleanedUspDf['TR_DATE_POTENCE'] =  pd.to_datetime(cleanedUspDf['TR_DATE_POTENCE'], format='%d.%m.%Y %H:%M')
cleanedUspDf['TREMPE_SK_TEMPS'] =  pd.to_datetime(cleanedUspDf['TREMPE_SK_TEMPS'], format='%Y%m%d')
cleanedUspDf['TREMPE_DATE_DEBUT'] =  pd.to_datetime(cleanedUspDf['TREMPE_DATE_DEBUT'], format='%Y%m%d%H%M%S')
cleanedUspDf['TREMPE_DATE_FIN'] =  pd.to_datetime(cleanedUspDf['TREMPE_DATE_FIN'], format='%Y%m%d%H%M%S')
cleanedUspDf['TRAC_SK_TEMPS'] =  pd.to_datetime(cleanedUspDf['TRAC_SK_TEMPS'], format='%Y%m%d')
cleanedUspDf['REVENU_DATE_DEBUT'] =  pd.to_datetime(cleanedUspDf['REVENU_DATE_DEBUT'], format='%Y%m%d%H%M%S')
cleanedUspDf['RASPTAB_SK_TEMPS'] =  pd.to_datetime(cleanedUspDf['RASPTAB_SK_TEMPS'], format='%Y%m%d')

In [421]:
# usp_features = ['ALLIAGE_INTERNE', 'FOUR_COULEE', 'PRECHAUFFAGE_NUMERO_FOUR', 'PRECHAUFFAGE_POSITION_FOUR', 'TREMPE_MACHINE', 'TREMPE_RECETTE', 'REVENU_MACHINE']
#usp_features = ['ALLIAGE_INTERNE', 'FOUR_COULEE', 'COMPO_SI', 'COMPO_FE', 'COMPO_CU', 'COMPO_MN', 'COMPO_MG', 'COMPO_CR', 'COMPO_NI', 'COMPO_ZN', 'COMPO_TI', 'COMPO_ZR', 'PRECHAUFFAGE_NUMERO_FOUR', 'PRECHAUFFAGE_POSITION_FOUR', 'PRECHAUFFAGE_T_C_AIR_MAXI', 'TR_DATE_POTENCE', 'TR_T_C_POTENCE', 'TR_EPAISSEUR_POTENCE', 'TR_NOMBRE_EBAUCHE', 'TR_NOMBRE_PASSE', 'TR_EPAISSEUR_DERNIERE_PASSE', 'TR_VITESSE_DERNIERE_PASSE', 'TR_T_C_DERNIERE_PASSE', 'TR_VITESSE_ENGAGEMENT_DERNIERE_PASSE', 'TR_TPS_POTENCE_PREMIERE_PASSE', 'TR_REDUCTION_EPAISSEUR_DERNIERE_PASSE', 'TR_REDUCTION_EPAISSEUR_AVANT_DERNIERE_PASSE', 'TR_REDUCTION_EPAISSEUR_AVANT_AVANT_DERNIERE_PASSE', 'TR_REDUCTION_EPAISSEUR_MOYENNE', 'BARRE_LONGUEUR', 'BARRE_LARGEUR', 'TREMPE_MACHINE', 'TREMPE_SK_TEMPS', 'TREMPE_DATE_DEBUT', 'TREMPE_DATE_FIN', 'TREMPE_RECETTE', 'TRAC_MACHINE', 'TRAC_SK_TEMPS', 'TRAC_TXPREML', 'TRAC_TXDERNL', 'TRAC_PLNTRANS', 'TRAC_PLNLONG', 'TRAC_PLNTRANSTETE', 'TRAC_PLNTRANSMILIEU', 'TRAC_PLNTRANSPIED', 'TRAC_EPMINL', 'TRAC_EPMAXL', 'REVENU_MACHINE', 'REVENU_DATE_DEBUT', 'REVENU_NOCHARGE']
usp_features = ['ALLIAGE_INTERNE', 'FOUR_COULEE', 'COMPO_SI', 'COMPO_FE', 'COMPO_CU', 'COMPO_MN', 'COMPO_MG', 'COMPO_CR', 'COMPO_NI', 'COMPO_ZN', 'COMPO_TI', 'COMPO_ZR', 'PRECHAUFFAGE_NUMERO_FOUR', 'PRECHAUFFAGE_POSITION_FOUR', 'PRECHAUFFAGE_T_C_AIR_MAXI', 'TR_T_C_POTENCE', 'TR_EPAISSEUR_POTENCE', 'TR_NOMBRE_EBAUCHE', 'TR_NOMBRE_PASSE', 'TR_EPAISSEUR_DERNIERE_PASSE', 'TR_VITESSE_DERNIERE_PASSE', 'TR_T_C_DERNIERE_PASSE', 'TR_VITESSE_ENGAGEMENT_DERNIERE_PASSE', 'TR_TPS_POTENCE_PREMIERE_PASSE', 'TR_REDUCTION_EPAISSEUR_DERNIERE_PASSE', 'TR_REDUCTION_EPAISSEUR_AVANT_DERNIERE_PASSE', 'TR_REDUCTION_EPAISSEUR_AVANT_AVANT_DERNIERE_PASSE', 'TR_REDUCTION_EPAISSEUR_MOYENNE', 'BARRE_LONGUEUR', 'BARRE_LARGEUR', 'TREMPE_MACHINE', 'TREMPE_RECETTE', 'TRAC_MACHINE', 'TRAC_TXPREML', 'TRAC_TXDERNL', 'TRAC_PLNTRANS', 'TRAC_PLNLONG', 'TRAC_PLNTRANSTETE', 'TRAC_PLNTRANSMILIEU', 'TRAC_PLNTRANSPIED', 'TRAC_EPMINL', 'TRAC_EPMAXL', 'REVENU_MACHINE', 'REVENU_NOCHARGE']
        
# By convention this is called X
X = filteredDf[usp_features]
X.head(1)

,ALLIAGE_INTERNE,FOUR_COULEE,COMPO_SI,COMPO_FE,COMPO_CU,COMPO_MN,COMPO_MG,COMPO_CR,COMPO_NI,COMPO_ZN,COMPO_TI,COMPO_ZR,PRECHAUFFAGE_NUMERO_FOUR,PRECHAUFFAGE_POSITION_FOUR,PRECHAUFFAGE_T_C_AIR_MAXI,TR_T_C_POTENCE,TR_EPAISSEUR_POTENCE,TR_NOMBRE_EBAUCHE,TR_NOMBRE_PASSE,TR_EPAISSEUR_DERNIERE_PASSE,TR_VITESSE_DERNIERE_PASSE,TR_T_C_DERNIERE_PASSE,TR_VITESSE_ENGAGEMENT_DERNIERE_PASSE,TR_TPS_POTENCE_PREMIERE_PASSE,TR_REDUCTION_EPAISSEUR_DERNIERE_PASSE,TR_REDUCTION_EPAISSEUR_AVANT_DERNIERE_PASSE,TR_REDUCTION_EPAISSEUR_AVANT_AVANT_DERNIERE_PASSE,TR_REDUCTION_EPAISSEUR_MOYENNE,BARRE_LONGUEUR,BARRE_LARGEUR,TREMPE_MACHINE,TREMPE_RECETTE,TRAC_MACHINE,TRAC_TXPREML,TRAC_TXDERNL,TRAC_PLNTRANS,TRAC_PLNLONG,TRAC_PLNTRANSTETE,TRAC_PLNTRANSMILIEU,TRAC_PLNTRANSPIED,TRAC_EPMINL,TRAC_EPMAXL,REVENU_MACHINE,REVENU_NOCHARGE
34,7215,HG_TP09,0.1,0.18,1.49,0.053,2.47,0.19,0.0053,5.74,0.027,0.017,F49,10,322.9,395,354.0,2,25,15.66,-1143.1,373.71,-530.3,104.12,3.95,4.95,6.08,13.53,2500,1650,FTRE06,R4675,TRAC04,1.7,1.7,2.7,0.8,0.0,0.0,0.0,0.0,0.0,FAIR09,20165009.0


In [422]:
X.describe()

,ALLIAGE_INTERNE,COMPO_SI,COMPO_FE,COMPO_CU,COMPO_MN,COMPO_MG,COMPO_CR,COMPO_NI,COMPO_ZN,COMPO_TI,COMPO_ZR,PRECHAUFFAGE_POSITION_FOUR,PRECHAUFFAGE_T_C_AIR_MAXI,TR_T_C_POTENCE,TR_EPAISSEUR_POTENCE,TR_NOMBRE_EBAUCHE,TR_NOMBRE_PASSE,TR_EPAISSEUR_DERNIERE_PASSE,TR_VITESSE_DERNIERE_PASSE,TR_T_C_DERNIERE_PASSE,TR_VITESSE_ENGAGEMENT_DERNIERE_PASSE,TR_TPS_POTENCE_PREMIERE_PASSE,TR_REDUCTION_EPAISSEUR_DERNIERE_PASSE,TR_REDUCTION_EPAISSEUR_AVANT_DERNIERE_PASSE,TR_REDUCTION_EPAISSEUR_AVANT_AVANT_DERNIERE_PASSE,TR_REDUCTION_EPAISSEUR_MOYENNE,BARRE_LONGUEUR,BARRE_LARGEUR,TRAC_TXPREML,TRAC_TXDERNL,TRAC_PLNTRANS,TRAC_PLNLONG,TRAC_PLNTRANSTETE,TRAC_PLNTRANSMILIEU,TRAC_PLNTRANSPIED,TRAC_EPMINL,TRAC_EPMAXL,REVENU_NOCHARGE
count,1284.000000,1284.000000,1284.000000,1284.000000,1284.000000,1284.000000,1284.000000,1284.000000,1284.000000,1284.000000,1284.000000,1284.000000,1284.000000,1284.000000,1284.000000,1284.000000,1284.000000,1284.000000,1284.000000,1284.000000,1284.000000,1284.000000,1284.000000,1284.000000,1284.000000,1284.000000,1284.000000,1284.000000,1284.000000,1284.000000,1284.000000,1284.000000,1284.0,1284.0,1284.0,1284.0,1284.0,1.284000e+03
mean,7214.831776,0.086509,0.149373,1.480359,0.050590,2.478754,0.185894,0.005211,5.819483,0.026231,0.017619,4.528816,377.412850,389.521028,353.909657,1.988318,24.390966,15.705740,-1100.858723,362.381768,-300.957165,168.381301,5.543115,6.775576,11.640880,13.893707,2547.741433,1622.196262,1.772352,1.764019,1.288551,0.846636,0.0,0.0,0.0,0.0,0.0,2.017270e+07
std,0.549685,0.010928,0.017014,0.046141,0.009318,0.075039,0.003767,0.001088,0.092725,0.003728,0.003773,2.595027,80.056647,7.258313,1.020262,0.107493,1.209128,0.318647,358.345319,17.873143,264.171628,81.636252,0.854565,1.375628,3.465517,0.554199,204.705842,78.033141,0.215510,0.273566,0.619381,0.336945,0.0,0.0,0.0,0.0,0.0,7.368247e+03
min,7213.000000,0.060000,0.110000,1.329900,0.027800,2.187100,0.180000,0.003000,5.284800,0.014000,0.010000,1.000000,203.500000,373.000000,349.000000,1.000000,23.000000,15.480000,-1199.900000,267.520000,-536.300000,21.950000,1.800000,1.810000,2.660000,10.250000,1800.000000,1400.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,2.016007e+07
25%,7215.000000,0.079700,0.139700,1.446800,0.044000,2.470000,0.182300,0.004500,5.781700,0.023800,0.015200,3.000000,317.800000,385.000000,354.000000,2.000000,24.000000,15.660000,-1153.800000,356.830000,-397.500000,86.570000,5.300000,5.700000,8.000000,13.570000,2500.000000,1650.000000,1.600000,1.600000,0.800000,0.600000,0.0,0.0,0.0,0.0,0.0,2.016542e+07
50%,7215.000000,0.087700,0.149500,1.480000,0.051200,2.492800,0.185900,0.005100,5.820000,0.027200,0.017400,4.000000,396.800000,390.000000,354.000000,2.000000,24.000000,15.670000,-1147.400000,365.440000,-390.600000,185.650000,5.490000,7.030000,13.490000,14.100000,2500.000000,1650.000000,1.700000,1.800000,1.000000,0.800000,0.0,0.0,0.0,0.0,0.0,2.017516e+07
75%,7215.000000,0.091200,0.160000,1.513300,0.058000,2.520000,0.190000,0.005600,5.867800,0.028425,0.019900,6.000000,451.000000,395.000000,354.000000,2.000000,25.000000,15.690000,-1134.400000,370.900000,-140.400000,239.960000,5.790000,7.300000,13.790000,14.100000,2700.000000,1650.000000,1.900000,1.900000,1.700000,1.000000,0.0,0.0,0.0,0.0,0.0,2.017563e+07
max,7215.000000,0.114500,0.180000,1.567800,0.074200,2.600000,0.191000,0.008200,6.000000,0.034700,0.029300,10.000000,510.000000,408.000000,356.000000,2.000000,33.000000,18.970000,2536.500000,417.140000,2093.600000,309.550000,9.850000,16.930000,19.290000,14.710000,2900.000000,1650.000000,2.500000,2.500000,4.800000,2.800000,0.0,0.0,0.0,0.0,0.0,2.019506e+07


### Encoding Categorical Features

In [423]:
cleanedUspDf.dtypes

OF                                      int64
BARRE                                   int64
ALLIAGE_INTERNE                         int64
EPAISSEUR_FINALE                      float64
FOUR_COULEE                            object
COMPO_SI                              float64
COMPO_FE                              float64
COMPO_CU                              float64
COMPO_MN                              float64
COMPO_MG                              float64
COMPO_CR                              float64
COMPO_NI                              float64
COMPO_ZN                              float64
COMPO_TI                              float64
COMPO_ZR                              float64
PRECHAUFFAGE_NUMERO_FOUR               object
PRECHAUFFAGE_POSITION_FOUR              int64
PRECHAUFFAGE_T_C_AIR_MAXI             float64
TR_DATE_POTENCE                datetime64[ns]
TR_T_C_POTENCE                          int64
TR_EPAISSEUR_POTENCE                  float64
TR_NOMBRE_EBAUCHE                 

In [424]:
# Get all object coloumns which need to be changes or encoded somehow
ObjectsXDf = X.loc[:, X.dtypes == object]
NonObjectsXDf = X.loc[:, X.dtypes != object]

In [425]:
# Columns need to be transformed
cols_to_transform = ObjectsXDf.columns
cols_to_transform

Index(['FOUR_COULEE', 'PRECHAUFFAGE_NUMERO_FOUR', 'TREMPE_MACHINE', 'TREMPE_RECETTE', 'TRAC_MACHINE', 'REVENU_MACHINE'], dtype='object')

In [426]:
# Columns to keep
cols_to_retain = NonObjectsXDf.columns
cat_dict = X[ cols_to_retain ].to_dict( orient = 'records' )
#cat_dict

In [427]:
# Pandas get.dummies not so good for ML
X = pd.get_dummies(X, columns = cols_to_transform )

#from sklearn.preprocessing import OneHotEncoder
#ohe = OneHotEncoder(sparse=True, handle_unknown="ignore")
#X = X.apply(ohe.fit(X[ cols_to_retain ]))

### Split Test Train

In [428]:
from sklearn.model_selection import train_test_split
len(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

## Build the model

In [452]:
runs = 1000

### DecisionTree Regressor

In [455]:
from sklearn.tree import DecisionTreeRegressor

# Define model. Specify a number for random_state to ensure same results each run
model = DecisionTreeRegressor(random_state=1)

result = []
run = 0
for run in range(runs):
    # Fit model
    model.fit(X_train, y_train)
    
    # Validate model
    result.append(model.score(X_test, y_test))

print("After {} runs\n  * min accuracy: {}\n  * max accuracy: {}".format(run, min(result)*100.0, max(result)*100.0))

After 999 runs
  * min accuracy: 70.01859345079046
  * max accuracy: 70.01859345079046


### RandomForest

In [456]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

model = RandomForestClassifier()
result = []
run = 0
for run in range(runs):
    # Fit model
    model.fit(X_train, y_train)
    
    # Validate model
    y_predict = model.predict(X_test)
    result.append(accuracy_score(y_test.values, y_predict))

print("After {} runs\n  * min accuracy: {}\n  * max accuracy: {}".format(run, min(result)*100.0, max(result)*100.0))

C:\Users\silvan.zahno\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.



After 999 runs
  * min accuracy: 49.532710280373834
  * max accuracy: 57.943925233644855
